##### Copyright 2020 The TensorFlow Authors.

In [7]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

# Additional Data Sources

"Leafsnap: A Computer Vision System for Automatic Plant Species Identification,"
Neeraj Kumar, Peter N. Belhumeur, Arijit Biswas, David W. Jacobs, W. John Kress, Ida C. Lopez, João V. B. Soares,
Proceedings of the 12th European Conference on Computer Vision (ECCV), October 2012

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [8]:
!pip install tflite-model-maker-nightly

Import the required packages.

In [9]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

## Simple End-to-End Example

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

In [10]:
import os
os.getcwd()

'/content'

In [13]:
import shutil
shutil.unpack_archive("flower_photos.zip")
image_path = os.path.join(os.path.dirname('flower_photos.zip'), 'flower_photos')

You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [14]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 7719, num_label: 184, labels: abies_concolor, abies_nordmanniana, acer_campestre, acer_ginnala, acer_griseum, acer_negundo, acer_palmatum, acer_pensylvanicum, acer_platanoides, acer_pseudoplatanus, acer_rubrum, acer_saccharinum, acer_saccharum, aesculus_flava, aesculus_glabra, aesculus_hippocastamon, aesculus_pavi, ailanthus_altissima, albizia_julibrissin, amelanchier_arborea, amelanchier_canadensis, amelanchier_laevis, asimina_triloba, betula_alleghaniensis, betula_jacqemontii, betula_lenta, betula_nigra, betula_populifolia, broussonettia_papyrifera, carpinus_betulus, carpinus_caroliniana, carya_cordiformis, carya_glabra, carya_ovata, carya_tomentosa, castanea_dentata, catalpa_bignonioides, catalpa_speciosa, cedrus_atlantica, cedrus_deodara, cedrus_libani, celtis_occidentalis, celtis_tenuifolia, cercidiphyllum_japonicum, cercis_canadensis, chamaecyparis_pisifera, chamaecyparis_thyoides, chionanthus_retusus, chionanthus_virginicus, cladrastis_lutea

2. Customize the TensorFlow model.

In [15]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 184)               235704    
Total params: 3,648,728
Trainable params: 235,704
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


217/217 [==============================] - 297s 1s/step - loss: 4.0517 - accuracy: 0.2468
Epoch 2/5
217/217 [==============================] - 301s 1s/step - loss: 2.6394 - accuracy: 0.5835
Epoch 3/5
217/217 [==============================] - 299s 1s/step - loss: 2.1397 - accuracy: 0.7092
Epoch 4/5
217/217 [==============================] - 297s 1s/step - loss: 1.8925 - accuracy: 0.7814
Epoch 5/5
217/217 [==============================] - 289s 1s/step - loss: 1.7492 - accuracy: 0.8165


3. Evaluate the model.

In [16]:
loss, accuracy = model.evaluate(test_data)

25/25 [==============================] - 41s 1s/step - loss: 1.7921 - accuracy: 0.7940


4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [17]:
model.export(export_dir='.', with_metadata=False)

INFO:tensorflow:Assets written to: /tmp/tmph7s2rh31/assets


INFO:tensorflow:Assets written to: /tmp/tmph7s2rh31/assets


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:Saving labels in ./labels.txt.


INFO:tensorflow:Saving labels in ./labels.txt.


After this simple 4 steps, we can now download the model and label files, and continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android/#4).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).
